# Black-Scholes Option Pricing with Hedgehog.jl

This notebook demonstrates how to use Hedgehog.jl to price options using the Black-Scholes model. Hedgehog is a Julia derivatives pricing library that implements a clean, SciML-inspired `solve(problem, method)` design pattern.

## Setup

In [ ]:
# Add Hedgehog.jl and Accessors
using Pkg
Pkg.add(url="https://github.com/aleCombi/Hedgehog.jl")
Pkg.add("Accessors")

using Hedgehog
using Accessors
using Dates
using Plots

## 1. Define Option Contract

We begin by defining a European Put Option expiring on January 1st, 2021. In Hedgehog, options are represented by `VanillaOption` objects that specify:
- Strike price
- Expiry date
- Exercise style (European or American)
- Option type (Call or Put)
- Underlying type (Spot or Forward)

In [ ]:
# Define payoff
strike = 1.0
expiry = Date(2021, 1, 1)
payoff = VanillaOption(strike, expiry, European(), Put(), Spot())

## 2. Specify Market Inputs

Next, we define market parameters needed to price the option:
- Reference date (valuation date)
- Risk-free interest rate
- Underlying spot price
- Volatility

The Black-Scholes model assumes constant volatility and interest rate.

In [ ]:
# Define market inputs
reference_date = Date(2020, 1, 1)
rate = 0.2       # 20% risk-free rate
spot = 1.0       # Spot price of 1.0
sigma = 0.4      # 40% volatility
market_inputs = BlackScholesInputs(reference_date, rate, spot, sigma)

## 3. Create Pricing Problem

In Hedgehog, a pricing problem combines a payoff definition with market inputs.

In [ ]:
pricing_problem = PricingProblem(payoff, market_inputs)

## 4. Price the Option

The Black-Scholes formula provides an analytical solution for European options. The formula for a put option is:

$$P = Ke^{-rT}N(-d_2) - S_0N(-d_1)$$

where:
- $d_1 = \frac{\ln(S_0/K) + (r + \sigma^2/2)T}{\sigma\sqrt{T}}$
- $d_2 = d_1 - \sigma\sqrt{T}$
- $N(x)$ is the standard normal cumulative distribution function

In [ ]:
analytical_price = solve(pricing_problem, BlackScholesAnalytic()).price
println("Black-Scholes analytical price: $analytical_price")

## 5. Alternative Pricing Method: Binomial Tree

We can verify our analytical solution using a Cox-Ross-Rubinstein binomial tree model. This numerical method approximates the continuous process with a discrete tree of possible values.

In [ ]:
# Cox-Ross-Rubinstein with 800 steps
steps = 800
crr = CoxRossRubinsteinMethod(steps)
crr_price = solve(pricing_problem, crr).price
println("Binomial tree price (800 steps): $crr_price")
println("Difference from analytical: $(abs(analytical_price - crr_price))")

# Increase steps for higher accuracy
steps = 10_000
crr = CoxRossRubinsteinMethod(steps)
crr_price = solve(pricing_problem, crr).price
println("Binomial tree price (10,000 steps): $crr_price")
println("Difference from analytical: $(abs(analytical_price - crr_price))")

## 6. Calculating Greeks

"Greeks" measure how option prices change with respect to various parameters. Using Hedgehog, we can calculate these sensitivities using multiple methods:

1. Finite differences
2. Automatic differentiation
3. Analytical formulas (when available)

### Using Lenses to Define Partial Derivatives

Hedgehog uses the concept of "lenses" from functional programming to elegantly specify which parameter we want to differentiate with respect to. A lens is essentially a getter/setter that allows us to focus on a specific field within a nested structure.

The [Accessors.jl](https://github.com/JuliaObjects/Accessors.jl) package provides these lenses in Julia, allowing us to:
- Precisely target specific parameters in complex data structures
- Create partial functions for differentiation
- Modify deeply nested fields without mutating the original structure

When calculating Greeks, we use lenses to tell Hedgehog which market parameter should be perturbed, effectively creating a partial derivative function. This approach makes the code both expressive and composable.

### Calculating Delta

Delta measures the rate of change of option price with respect to changes in the underlying asset's price:

$$\Delta = \frac{\partial V}{\partial S}$$

For a put option in the Black-Scholes model:
$$\Delta_{put} = N(d_1) - 1$$

We use the `@optic` macro from Accessors.jl to create a lens targeting the spot price:

In [ ]:
# Define lens for spot price - this creates a partial function for differentiation
spot_lens = @optic _.market_inputs.spot
greek_problem = GreekProblem(pricing_problem, spot_lens)

# Calculate delta using three methods
delta_fd = solve(greek_problem, FiniteDifference(1e-3), BlackScholesAnalytic()).greek
delta_ad = solve(greek_problem, ForwardAD(), BlackScholesAnalytic()).greek
delta_an = solve(greek_problem, AnalyticGreek(), BlackScholesAnalytic()).greek

println("Delta (Finite Difference): $delta_fd")
println("Delta (Auto Diff): $delta_ad")
println("Delta (Analytical): $delta_an")

### Calculating Vega

Vega measures sensitivity to volatility:

$$\nu = \frac{\partial V}{\partial \sigma}$$

For both calls and puts in the Black-Scholes model:
$$\nu = S_0 \sqrt{T} \cdot n(d_1)$$

where $n(x)$ is the standard normal probability density function.

For volatility, Hedgehog provides a specialized `VolLens` since volatility can be either a scalar value or part of a more complex surface structure. This lens enables us to target specific volatility pillars when working with volatility surfaces:

In [ ]:
# Define lens for volatility - VolLens(i,j) targets the (i,j) element in a vol surface
# For flat volatility surfaces, we simply use VolLens(1,1)
vol_lens = VolLens(1,1)
greek_problem = GreekProblem(pricing_problem, vol_lens)

# Calculate vega using three methods
vega_fd = solve(greek_problem, FiniteDifference(1e-3), BlackScholesAnalytic()).greek
vega_ad = solve(greek_problem, ForwardAD(), BlackScholesAnalytic()).greek
vega_an = solve(greek_problem, AnalyticGreek(), BlackScholesAnalytic()).greek

println("Vega (Finite Difference): $vega_fd")
println("Vega (Auto Diff): $vega_ad")
println("Vega (Analytical): $vega_an")

## 7. Batch Greek Calculation

Often, we want to calculate multiple Greeks at once. Hedgehog provides a `BatchGreekProblem` for efficient calculation of multiple sensitivities:

In [ ]:
# Calculate delta and vega in one call
batch_prob = BatchGreekProblem(pricing_problem, [vol_lens, spot_lens])
greeks = solve(batch_prob, ForwardAD(), BlackScholesAnalytic())

println("Batch Greek Results:")
for (greek_name, value) in greeks
    println("  $greek_name: $value")
end

## 8. Implied Volatility Calculation

Implied volatility is the volatility value that makes the Black-Scholes price match a market price. This is effectively solving the inverse problem.

In [ ]:
# Setup calibration problem - we'll use our analytical price as "market price"
market_price = analytical_price
basket_problem = BasketPricingProblem([payoff], market_inputs)
calib = CalibrationProblem(
    basket_problem, 
    BlackScholesAnalytic(), 
    [vol_lens], 
    [market_price], 
    [0.2]  # Initial guess of 20% volatility
)

# Solve using root finder
result = solve(calib, RootFinderAlgo())
println("Target price: $market_price")
println("Initial volatility guess: 20%")
println("Implied volatility: $(result.u[1] * 100)%")
println("True volatility used in model: $(sigma * 100)%")

## Summary

This notebook demonstrated Hedgehog.jl's capabilities for option pricing:

1. Creating option contracts and market environment
2. Analytical Black-Scholes pricing
3. Numerical pricing with binomial trees
4. Greeks calculation using multiple methods
5. Implied volatility calculation

Hedgehog's composable, SciML-inspired design makes it easy to swap between different pricing methods and parameter settings while maintaining a consistent interface.